In [ ]:
# =====================================================================================
# ====================================== IMPORTANDO AS BIBLIOTECAS QUE SERÃO UTILIZADAS

from selenium.webdriver.support import expected_conditions as EC # type: ignore
from selenium.common.exceptions import WebDriverException # type: ignore
from selenium.webdriver.support.ui import WebDriverWait # type: ignore
from selenium.webdriver.common.alert import Alert # type: ignore
from selenium.webdriver.support.ui import Select # type: ignore
from selenium.webdriver.common.by import By # type: ignore
from selenium import webdriver # type: ignore
from bs4 import BeautifulSoup # type: ignore
from time import sleep # type: ignore
from credenciais import credenciais


import tkinter as tk  # type: ignore
import pandas as pd # type: ignore 
import threading # type: ignore
import keyboard # type: ignore
import os # type: ignore

In [ ]:
def acessar_iframe(driver, xpaths):
    """
        Mergulha em uma lista de iframes, com base nos seus XPATHs.

        Args:
            driver: Instância do Selenium WebDriver
            xpaths: Lista com os XPATHs dos iframes a serem mergulhados

        Returns:
            None
    """
    for xpath in xpaths:
        iframe = driver.find_element(By.XPATH, xpath)
        driver.switch_to.frame(iframe)

In [ ]:
def esperar_elemento(driver, by, endereco):
    """
    Aguarda até que um elemento esteja visível.

    Args:
        driver: Instância do WebDriver.
        by: Estratégia de localizador (e.g., By.ID, By.XPATH, ...).
        endereco: Valor do localizador para o elemento.

        "e.g." é uma abreviação do latim "exempli gratia", que significa "por exemplo"
    
    Returns:
        WebElement se estiver clicável dentro do período de tempo limite.
    """
    max_tentativas = 10
    num_tentativas = 0

    while num_tentativas <= max_tentativas:
            elemento = WebDriverWait(driver, 20).until(EC.presence_of_element_located((by, endereco)))

            if elemento:
                return elemento
            else:
                num_tentativas += 1
                sleep(1)

In [ ]:
def janela_confirmar_filtros():
    """
        Função para mostrar uma janela de confirmação

        Returns:
            True: valor booleano 

            Ao clicar em "OK", é retornado True.
    """

    janela = tk.Tk() # Cria a janela principal
    janela.withdraw() # Oculta as janela possíveis janelas
    janela.title("Confirmação") # Altera o título da janela
    janela.geometry("300x100") # Ajusta o tamanho da janela
    janela.resizable(width=False, height=False) # Bloqueia o redimensionamento da janela
    janela.configure(bg='lightblue') # Altera a cor de fundo da janela
    # janela.iconphoto(True, tk.PhotoImage(file='Py.jpg')) # Altera o ícone da janela

    # Cria um rótulo com a mensagem
    mensagem = tk.Label(janela, text=f'Ao finalizar pressione o botão:\n{"↓":^30}', font=('Helvetica', 12))
    mensagem.pack(pady=10) # Adiciona um espaçamento acima e abaixo do rótulo
    
    # Cria um botão e centraliza
    botao_ok = tk.Button(janela, text="OK", command=janela.destroy, font=('Helvetica', 12))
    botao_ok.pack(pady=10) 
    
    janela.eval('tk::PlaceWindow . center') # Centraliza a janela na tela
    
    janela.deiconify() # Mostrar a janela novamente
    janela.mainloop() # Inicia o loop principal da janela

    return True

In [ ]:
def obter_valores_filtros(driver):

    # Xpaths dos filtros
    xpaths_elementos = [
        '//*[@id="cbFiltroMOD"]',
        '//*[@id="cbFiltroYEAR_REVERSE"]',
        '//*[@id="cbFiltroGRADE"]',
        '//*[@id="cbFiltroAREA"]',
        '//*[@id="cbFiltroORIGIN"]',
        '//*[@id="cbFiltroCOLOR"]'
    ]

    # Lista para receber os dados
    dados_filtros = []

    # Para cada "FILTRO" e "ENDEREÇO" do elemento
    for xpath in xpaths_elementos:
        # Localizando todos os "ELEMENTOS SELECT"
        tags_select = driver.find_elements(By.XPATH, xpath)

        # Para cada "ELEMENTO SELECT" dentro das "TAGS SELECT"
        for select in tags_select:
            select_element = Select(select)
            selected_options = select_element.all_selected_options

            if selected_options:
                valores = [option.text for option in selected_options]
                # Assumindo que o primeiro valor é o correto
                dados_filtros.append((valores[0]))
                break
    
    return dados_filtros

In [ ]:
def janela_info_pecas(driver):
    driver.switch_to.default_content()
    sleep(0.2)

    xpaths_iframes = [
        '//*[@id="content4"]',
        '//*[@id="ilustLista"]',
        '//*[@id="gridLista"]'
    ]

    acessar_iframe(driver, xpaths_iframes)
    sleep(0.2)

    tabela = driver.find_element(By.XPATH, '//*[@id="tabela"]')
    
    return tabela

In [ ]:
def formatar_dados(pagina_impressao):
    """
    Recebe um objeto "pagina_impressao" e retorna uma lista com o título do cabeçalho,
    uma lista com as linhas formatadas e a lista de textos da página.
    
    Args:
        pagina_impressao (object): Objeto com a página impressa.
    
    Returns:
        titulo_cabecalho (list): Lista com o título do cabeçalho.
        conteudo (list): Lista com as linhas formatadas.
    """
    
    html = BeautifulSoup(pagina_impressao.get_attribute('innerHTML'), 'html.parser')

    # Itens a serem removidos
    itens_a_remover = [
        '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
        'Info/TMO',
        'INMETRO',
        'Série início',
        'Série fim',
        'Modif.'
    ]


    # ===============================================
    # ===================================== CABEÇALHO

    tags_th = html.find_all('th')
    cabecalho = [
        coluna.text.strip() 
        for coluna in tags_th 
        if coluna.text.strip() 
        not in itens_a_remover
    ]


    # ===============================================
    # ====================================== CONTEÚDO

    # Encontra todas as tags <tr>
    linhas = html.find_all('tr')
    conteudo = []

    for i, linha in enumerate(linhas[1:]):
        col_1 = linha.find('td', {'id': f'c{i}_1'})
        col_2 = linha.find('td', {'id': f'c{i}_2'})
        col_4 = linha.find('td', {'id': f'c{i}_4'})
        col_5 = linha.find('td', {'id': f'c{i}_5'})

        if col_1 and col_2 and col_4 and col_5:
            ref = col_1.text
            cod_peca = col_2.text
            descricao = col_4.text
            qtd = col_5.text

            if ';' in qtd:
                qtd = '001'

            else:
                linha_formatada = f'{int(ref)}| {cod_peca}| {descricao}| {str(qtd)}'
                conteudo.append(linha_formatada)

    return cabecalho, conteudo

In [ ]:
def obetr_url_imagem(driver):
    # =====================================================================================
    # ======================= MERGULANDO NOS IFRAMES PARA OBTER O URL DA IMAGEM DO CATÁLOGO

    # Saindo do "IFRAME - CONTENT3"
    driver.switch_to.default_content()

    sleep(0.2)

    xpaths_iframes = [
        '//*[@id="content4"]',
        '//*[@id="ilustLista"]',
        '//*[@id="aux"]',
        '//*[@id="ilust"]'
    ]

    acessar_iframe(driver, xpaths_iframes) # type: ignore


# =====================================================================================
# ================================================= OBTENDO A URL DA IMAGEM DO CATÁLOGO

    tag_img = driver.find_element(By.XPATH, '//*[@id="imagem"]')
    sleep(1)
    url_imagem = tag_img.get_attribute('src')


    return url_imagem

In [ ]:
def salvar_arquivo(i, valores_filtros, url_imagem, conteudo, nome_catalogo, codigo_catalogo):
    """
    Salva os dados em um arquivo CSV, utilizando "pipe-line" como delimitador.

    Args:
        i: Índice do catálogo.
        valores_filtros: Lista com os valores dos filtros.
        url_imagem: URL da imagem do catálogo.
        conteudo: Dados da página de impressão formatados.
        nome_catalogo: Nomes dos catálogos.
        codigo_catalogo: Códigos dos catálogos.
    """

    # Desempacotando valores_filtros
    modelo, ano, versao, mercado_area, pais, cor = valores_filtros

    # Configurando o nome do arquivo CSV
    nome_arquivo = f'Arquivo Honda - {modelo} - {ano} - {versao} - {mercado_area} - {pais} - {cor.replace("*", "")}.csv'

    # Definindo o cabeçalho
    cabecalho = [
        'Index', 'Versão', 'Ano', 'Cor', 'Modelo', 'Código', 'Nome', 'Ref',
        'Código Peça', 'Descrição', 'Qtd', 'URLImage'
    ]

    # Criando a lista de linhas
    linhas = []
    for linha in conteudo:
        partes = linha.split('|')
        if len(partes) == 4:
            ref, cod_peca, descricao, qtd = partes
            linhas.append([
                f"0{i+1:02d}",  
                versao,
                ano,
                cor,
                modelo,
                codigo_catalogo.strip(),
                nome_catalogo.strip(),
                ref.strip(),
                cod_peca.strip(),
                descricao.strip(),
                qtd.strip(),
                url_imagem
            ])
        else:
            print(f"Erro ao processar a linha: {linha}. Esperado 4 partes, mas obtido {len(partes)}.")

    # Verificando se há linhas para salvar
    if linhas:
        # Criando o DataFrame
        df = pd.DataFrame(linhas, columns=cabecalho)

        # Salvando o DataFrame no arquivo CSV com delimitador "|"
        if not os.path.isfile(nome_arquivo):
            df.to_csv(nome_arquivo, index=False, sep='|', encoding='utf-8')
        else:
            df.to_csv(nome_arquivo, mode='a', header=False, index=False, sep='|', encoding='utf-8')
    else:
        print("Nenhuma linha para salvar no arquivo CSV.")

In [ ]:
def pagina_inicial(driver):
    # Retorna para a página inicial
    driver.switch_to.default_content()
    sleep(0.3)

    xpaths_iframes = [
        '//*[@id="content8"]',
        '//*[@id="frTotem"]'
    ]

    acessar_iframe(driver, xpaths_iframes)

    btn_honda = driver.find_element(By.XPATH, '//*[@id="logo_text"]')
    sleep(0.2)
    btn_honda.click()


    driver.switch_to.default_content()
    sleep(1)

    # XPATHS dos filtros
    iframe_level_1 = driver.find_element(By.XPATH, '//*[@id="content5"]')
    driver.switch_to.frame(iframe_level_1)

    sleep(1)

In [ ]:
def extrair_nomes_catalogos(conteudo_html):
    """
    Extrai códigos e nomes de catálogos de uma página HTML.

    Dado o conteúdo HTML de uma página, essa função extrai os códigos e nomes
    de todos os catálogos presentes na página e os retorna em duas listas
    separadas.

    Args:
        conteudo_html (str): Conteúdo HTML da página.

    Returns:
        tuple: Duas listas, a primeira lista contém os códigos dos catálogos e
        a segunda lista contém os nomes dos catálogos.
    """
    sopa = BeautifulSoup(conteudo_html, 'html.parser')

    # Encontra o elemento <tr> específico com id="row0"
    linha_inicio = sopa.find('tr', id='row0')

    # Inicializa listas para armazenar códigos e nomes de catálogos
    codigos_catalogos = []
    nomes_catalogos = []

    # Itera sobre todos os elementos <tr> a partir do elemento encontrado
    linha_atual = linha_inicio

    while linha_atual:
        celulas = linha_atual.find_all('td')

        # Verifica se houver pelo menos duas celulas
        if len(celulas) >= 2:
            # Extrai códigos e nomes dos catálogos
            codigo = celulas[0].text.strip()
            nome = celulas[1].text.strip()

            # Adiciona os códigos e nomes aos respectivos listas
            codigos_catalogos.append(codigo)
            nomes_catalogos.append(nome)

        # Move para a linha seguinte
        linha_atual = linha_atual.find_next_sibling('tr')

    return codigos_catalogos, nomes_catalogos

In [ ]:
def pagina_catalogo_pecas(driver):
# =====================================================================================
# =================================== JANELA PARA ESPERAR O USUÁRIO ESCOLHER OS FILTROS

    botao_ok = janela_confirmar_filtros() # Retorna True

    while True:
        if botao_ok:
            sleep(1)
            break


# =====================================================================================
# ========= SAINDO DO "IFRAME - CONTENT2" PARA O "IFRAME - CONTENT3 - MENU DOS FILTROS"

    # Saindo do "IFRAME CONTENT2" para o contexto do HTML
    driver.switch_to.default_content()
    sleep(1)

    # XPATHS dos filtros
    xpaths_iframes = (
        '//*[@id="content3"]', # Mergulhando na 1ª camada: "IFRAME - CONTENT3" - Página
        '//*[@id="frTotem"]' # Mergulhando na 2ª camada: "IFRAME FRTOTEM"
        )

    acessar_iframe(driver, xpaths_iframes)


# =====================================================================================
# ==================================================== OBTENDO OS "VALORES" DOS FILTROS

    # Armazanando os valores dos filtros em uma variável
    valores_filtros = obter_valores_filtros(driver)


# =====================================================================================
# =========================================== ACESSANDO CATÁLOGO DOS FILTROS ESCOLHIDOS

    # Saindo do "IFRAME - FRTOTEM" para o contexto do "IFRAME - CONTENT3"
    driver.switch_to.parent_frame()

    # Localizando o elemento do "1º CATÁLOGO" e clicando nele
    img_catalogo = esperar_elemento(driver, By.XPATH, '//*[@id="t0imgSubgrupo_text"]')
    sleep(0.5)
    img_catalogo.click()



# =====================================================================================
# ============================== MERGULHANDO NOS IFRAMES PARA OBTER NOMES DOS CATÁLOGOS

    # Saindo da "4ª CAMADA - IFRAME ILUST" e indo para a "3ª CAMADA - AUX"
    driver.switch_to.parent_frame()

    # XPATH do Iframe
    xpaths_iframes = [
        '//*[@id="content4"]',
        '//*[@id="ilustLista"]',
        '//*[@id="aux"]',
        '//*[@id="gridSG"]' # Mergulhando na 4ª camada: "IFRAME GRIDSG"
        ]

    acessar_iframe(driver, xpaths_iframes)

    return valores_filtros

In [ ]:
def proximo_catalogo(driver):

    # Saindo do "IFRAME - GRIDSG" para o contexto do HTML
    driver.switch_to.default_content()

    sleep(0.5)

    iframe = driver.find_element(By.XPATH, '//*[@id="content4"]')
    driver.switch_to.frame(iframe)

    # Localizando o elemento do "2º CATÁLOGO" e clicando nele
    btn_proximo = esperar_elemento(driver, By.XPATH, '//*[@id="btAvancar_text"]')
    sleep(0.2)
    btn_proximo.click()

In [ ]:
def extrair_e_salvar_dados_catalogo(driver):
    """ Extrai e salva os dados do catálogo de peças.
    
    Args:
        driver: Instância do WebDriver para controle do navegador.
    
    """

    # =====================================================================================
    # ================================= SCRIPT PRINCIPAL ==================================
    valores_filtros = pagina_catalogo_pecas(driver)


    # =====================================================================================
    # ======================================= IDENTIFICANDO TABELA COM NOME DOS CATÁLOGOS
    tabela_catalogos = esperar_elemento(driver, By.XPATH, '//*[@id="tabela"]')
    conteudo_html = tabela_catalogos.get_attribute('innerHTML')
    codigos_catalogo, nomes_catalogo = extrair_nomes_catalogos(conteudo_html)

    sleep(0.5)
            

    # =====================================================================================
    # ====================================== VERIFICANDO QUANTIDADE DE PÁGINAS DE IMPRESSÃO


    for i, catalogo in enumerate(nomes_catalogo):
        url_imagem = obetr_url_imagem(driver)
        pagina = janela_info_pecas(driver)
        cabecalho, conteudo = formatar_dados(pagina)
        print(f"URL da Imagem para {nomes_catalogo[i]}: {url_imagem}")

        # Passando "nomes_catalogo[i]" para a função "salvar_arquivo"
        salvar_arquivo(
            i,
            valores_filtros,
            url_imagem,
            conteudo,
            catalogo,
            codigos_catalogo[i]
        )

        sleep(1)

        proximo_catalogo(driver)

    # Retorna o elemento "Pop-Up"
    return Alert(driver)

In [ ]:
# =====================================================================================
# ====================== ABRINDO O NAVEGADOR AUTOMATIZADO E ACESSANDO A PÁGINA DE LOGIN

try:
    cpf, cpf_formatado, senha, pagina_web = credenciais.values()
    sleep(0.2)

    driver = webdriver.Chrome()
    driver.maximize_window()  
    driver.get(pagina_web)

except WebDriverException as erro:
    print(f'Ocorreu um erro: {erro.__class__.__name__}') # Mostra o nome do erro gerado


# =====================================================================================
# ============================================= PÁGINA PORTAL - PREENCHENDO CREDENCIAIS

# Armazena o identificador da "ABA ATUAL - PÁGINA PORTAL"
pagina_portal = driver.current_window_handle

# Espera o campo de "CPF" estar disponível na página para preenchê-lo
campo_login = esperar_elemento(driver, By.XPATH, '//*[@id="CPF"]')
campo_login.send_keys(cpf), sleep(1)

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

while True:
    # Obtém texto do "ATRIBUTO - VALUE"
    value_login = campo_login.get_attribute('value')

    if value_login == cpf_formatado:
        break
    else:
        # Simula o comando "CTRL + A" e espera 1 segundo
        driver.execute_script('arguments[0].select();', campo_login)
        sleep(4)
        campo_login.send_keys(cpf)
        sleep(0.2)

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =

# Obtendo os campos do formulário Senha
campo_senha = esperar_elemento(driver, By.XPATH, '//*[@id="Senha"]')
campo_senha.send_keys(senha)
sleep(0.2)

# Identificando e clicando no botão Acessar
botao_acessar = esperar_elemento(driver, By.XPATH, '//input[@value="acessar"]')
botao_acessar.click()


# =====================================================================================
# ================================================ PÁGINA PORTAL - ACESSANDO O CATÁLOGO

# Dentro do Portal "Pós-Venda", clica em "Peças"
menu_pecas = esperar_elemento(driver, By.XPATH, '//*[@id="qm0"]/a[4]')
menu_pecas.click()
sleep(2)

# Localiza "Catálogo de Peças" e clique nele
sub_menu_catalogo_pecas = esperar_elemento(driver, By.XPATH, '//*[@id="LinkCatalogoPecas"]')
sub_menu_catalogo_pecas.click()
sleep(3)

# =====================================================================================
# =============== FECHANDO ABA - PÁGINA PORTAL E ACESSANDO - PÁGINA CATÁLOGO ELETRÔNICO

# Obtém o identificador de "TODAS" abas abertas do navegador automatizado
for janela in driver.window_handles: # DIRETAMENTE
    if janela != pagina_portal:
        driver.close() # Fecha somente a aba    
        break
    
    sleep(2)
    
# Obtém o identificador de "TODAS" abas abertas do navegador automatizado
for janela in driver.window_handles:  # DIRETAMENTE
    driver.switch_to.window(janela)

    # Espera o "IFRAME - CONTENT2" ficar visível
    iframe_level_1 = esperar_elemento(driver, By.XPATH, '//*[@id="content2"]')
    
    if iframe_level_1:
        break

driver.switch_to.frame(iframe_level_1)

In [ ]:
# =====================================================================================
# ====================================================== PROCESSO DE EXTRAÇÃO DOS DADOS

while True:
    pop_up = extrair_e_salvar_dados_catalogo(driver)
    sleep(0.5)
    pop_up.accept()
    sleep(1)


# =====================================================================================
# ================================================================== CLICANDO EM VOLTAR
    # Saindo para o contexto do HTML
    driver.switch_to.default_content()

    xpaths_iframes = [
        '//*[@id="content4"]', # Mergulhando na 1ª camada: "IFRAME - CONTENT3" - Página
        '//*[@id="frTotem"]' # Mergulhando na 2ª camada: "IFRAME FRTOTEM"
        ]

    acessar_iframe(driver, xpaths_iframes)

    # LOCALIZANDO O "BOTÃO DE VOLTAR" E CLICANDO NELE
    btn_voltar = driver.find_element(By.XPATH, '//*[@id="buttonBar"]/*[@id="btVoltar"]')
    sleep(0.2)
    btn_voltar.click()

    sleep(2)

In [ ]:
# # =====================================================================================
# # ========================================================== CLICANDO NO BOTÃO "VOLTAR" 

# # Saindo para o contexto do HTML
# driver.switch_to.default_content()

# xpaths_iframes = [
#     '//*[@id="content4"]', # Mergulhando na 1ª camada: "IFRAME - CONTENT3" - Página
#     '//*[@id="frTotem"]' # Mergulhando na 2ª camada: "IFRAME FRTOTEM"
#     ]

# acessar_iframe(driver, xpaths_iframes)


# # LOCALIZANDO O "BOTÃO DE VOLTAR" E CLICANDO NELE
# btn_voltar = driver.find_element(By.XPATH, '//*[@id="buttonBar"]/*[@id="btVoltar"]')
# sleep(0.2)
# btn_voltar.click()

# sleep(2)


# # =====================================================================================
# # ==================================================== SELECIONANDO VALORES "VER TODOS" 
 
# driver.switch_to.default_content()

# xpaths_iframes = [
#     '//*[@id="content3"]', # Mergulhando na 1ª camada: "IFRAME - CONTENT3" - Página
#     '//*[@id="frTotem"]' # Mergulhando na 2ª camada: "IFRAME FRTOTEM"
#     ]

# acessar_iframe(driver, xpaths_iframes)

# sleep(0.2)

# filtros = [
#     '//*[@id="cbFiltroCOLOR"]', # --------→ FILTRO DE COR
#     '//*[@id="cbFiltroYEAR_REVERSE"]'  # -→ FILTRO DE ANO
# ]

# for filtro in filtros:
#     elemento = driver.find_element(By.XPATH, filtro)
#     valor = Select(elemento)
#     sleep(0.2)

#     valor.select_by_index(0)
#     sleep(0.5)

In [ ]:
# def opcoes_filtros(filtro):
#     return Select(filtro).options


# # Conjunto para armazenar combinações únicas
# combinacoes_processadas = set()

# campo_modelo = esperar_elemento(driver, By.XPATH, '//*[@id="cbFiltroMOD"]')
# sleep(0.2)

# modelos = opcoes_filtros(campo_modelo)






# # Identificando o "CAMPO DE ANO"
# campo_ano = esperar_elemento(driver, By.XPATH, '//*[@id="cbFiltroYEAR_REVERSE"]')
# sleep(0.2)

# # Obtendo todas as opções dentro do filtro "ANO"
# anos = opcoes_filtros(campo_ano)

# # Percorrendo sobre cada ano do filtro
# for i in range(1, len(anos)):
#     anos[i].click()
#     sleep(0.4)

#     campo_versao = esperar_elemento(driver, By.XPATH, '//*[@id="cbFiltroGRADE"]')
#     sleep(0.2)

#     versoes = opcoes_filtros(campo_versao)
#     if len(versoes) == 1:
#         continue
#     else:
#         for j in range(1, len(versoes)):
#             versoes[j].click()
#             sleep(0.4)

#             campo_mercado_area = esperar_elemento(driver, By.XPATH, '//*[@id="cbFiltroAREA"]')
#             sleep(0.2)

#             mercados_areas = opcoes_filtros(campo_mercado_area)
#             if len(mercados_areas) == 1:
#                 continue
#             else:
#                 for k in range(1, len(mercados_areas)):
#                     mercados_areas[k].click()
#                     sleep(0.4)

#                     campo_pais = esperar_elemento(driver, By.XPATH, '//*[@id="cbFiltroORIGIN"]')
#                     sleep(0.2)

#                     paises = opcoes_filtros(campo_pais)
#                     if len(paises) == 1:
#                         continue
#                     else:
#                         for l in range(1, len(paises)):
#                             paises[l].click()
#                             sleep(0.4)

#                             campo_cor = esperar_elemento(driver, By.XPATH, '//*[@id="cbFiltroCOLOR"]')
#                             sleep(0.2)

#                             cores = opcoes_filtros(campo_cor)
#                             if len(cores) == 1:
#                                 continue
#                             else:
#                                 for m in range(1, len(cores)):
#                                     cores[m].click()
#                                     sleep(0.4)

#                                     # Capturar a combinação atual
#                                     combinacao_atual = (anos[i].text, versoes[j].text, mercados_areas[k].text, paises[l].text, cores[m].text)

#                                     # Verificar se a combinação já foi processada
#                                     if combinacao_atual in combinacoes_processadas:
#                                         continue

#                                     # Adicionar a combinação atual ao conjunto
#                                     combinacoes_processadas.add(combinacao_atual)

#                                     pop_up = extrair_e_salvar_dados_catalogo(driver)
